In [1]:
:set -XNoMonadFailDesugaring
:l SortersM
:l Predicates

In [2]:
bubbleSortM coinCmp [3,2,1] :: [[Int]]

[[3,2,1],[3,1,2],[2,3,1],[2,1,3],[3,1,2],[3,2,1],[1,3,2],[1,2,3]]

In [3]:
import Control.Monad.State
evalStateT (bubbleSortM consistentCoinCmp [3,2,1]) noChoices :: [[Int]]

[[3,2,1],[2,3,1],[2,1,3],[3,1,2],[3,2,1],[1,3,2],[1,2,3]]

In [4]:
import Data.List (nub)
totalClosure :: (Eq a) => Choices a -> Choices a
totalClosure cs = nub $ cs ++ concatMap add cs
    where add ((x, y), b) = [((y, x), True) | not b]

In [5]:
totalConsistentCoinCmp :: (Eq a, MonadPlus m) => CmpMS a m
totalConsistentCoinCmp = checkChoices $ storeChoice totalClosure coinCmp

In [6]:
evalStateT (selectSortM totalConsistentCoinCmp [3,2,1]) noChoices :: [[Int]]

[[3,2,1],[2,3,1],[2,1,3],[3,1,2],[1,3,2],[1,2,3]]

In [7]:
bubbleM :: (Monad m) => CmpM a m -> [a] -> m [a]
bubbleM _ [] = return []
bubbleM _ [x] = return [x]
bubbleM p (x:xs) = do
    yss@(y:ys) <- bubbleM p xs
    b <- p x y
    return (if b then x : yss else y : x : ys)

bubbleSortM :: (Monad m) => SorterM a m
bubbleSortM _ [] = return []
bubbleSortM p xs = do
    (y:ys) <- bubbleM p xs
    fmap (y:) (bubbleSortM p ys)

evalStateT (bubbleSortM consistentCoinCmp [3,2,1]) noChoices :: [[Int]]

[[3,2,1],[2,3,1],[2,1,3],[3,1,2],[3,2,1],[1,3,2],[1,2,3]]

In [8]:
bubbleM :: (Monad m) => CmpM a m -> [a] -> m [a]
bubbleM _ [] = return []
bubbleM _ [x] = return [x]
bubbleM p (x:xs) = do
    (y:ys) <- bubbleM p xs
    b <- p x y
    return (if b then x : xs else y : x : ys)

bubbleSortM :: (Monad m) => SorterM a m
bubbleSortM _ [] = return []
bubbleSortM p xs = do
    (y:ys) <- bubbleM p xs
    fmap (y:) (bubbleSortM p ys)

evalStateT (bubbleSortM consistentCoinCmp [3,2,1]) noChoices :: [[Int]]

[[3,2,1],[2,3,1],[2,1,3],[3,1,2],[1,3,2],[1,2,3]]